#  Несмещённый и состоятельный записыватель курсов


Филипп записывает онлайн-курс по статистике. Он ориентируется на то, что каждая неделя курса должна идти около $100$ минут.
В голове у Филиппа есть оцениватель времени. Он оценивает по длительности каждое видео и записыват свои прикидки на бумагу.
Когда видео снято, Филипп смотрит как долго оно длилось и записывает итоговый результат на бумагу.
В таблице `filipp_course.tsv` представлены записи Филиппа за $11$ отснятых недель. 

__Описание колонок:__

- `real_week` - хронологический номер недели
- `week` - какой по порядку записывалась неделя
- `type` - тип видео (доска, презентация, код)
- `estimate_time` - во сколько минут Филипп оценил время видео при разработке материала
- `real_time` - сколько реально длится это видео

In [ ]:
!wget 'https://drive.google.com/u/0/uc?id=1KBcPTXykHBlcKP-tiH5ljLQxYaAbLmNE' -O "filipp_course.tsv"

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("filipp_course.tsv", sep='\t')
print(df.shape)
df.head()

(129, 5)


   real_week  week          type  estimate_time  real_time
0          6     1    blackboard              6       11.0
1          6     1    blackboard             10        7.0
2          6     1    blackboard             10       11.5
3          6     1    blackboard              5        6.0
4          6     1  presentation              5        3.0

Будем считать, что на протяжении всей записи у Филиппа в голове был один и тот же оцениватель (estimator).
По этому оценивателю Филипп делал точечные оценки для каждого видео независимо друг от друга.
Давайте поисследуем оцениватель Филиппа на различные статистические свойства.

__а)__ Посчитайте разницу между `estimate_time` и `real_time`. Запишите её в новую колонку `bias`.
Это то, насколько каждая оценка Филиппа оказалась смещена. Посчитайте среднее смещение, запишите его в переменную `bias_mean`.
Проинтерпретируйте получившееся число. Правда ли, что Филипп делает несмещённые оценки?

In [2]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

bias = df["estimate_time"] - df["real_time"]
bias_mean = bias.mean()
bias_mean

-2.1790697674418604

Неправда, оценки Филиппа сильно смещены от реальных значений.

In [ ]:
# проверка того, что задание решено верно (задание необязательно - все тесты открыты)
assert np.abs(bias_mean  + 2.179) < 1e-2

__б)__ Посчитайте среднее смещение, которое делает Филипп для каждого типа контента.
Для какого типа смещение оказывается самым низким? Запишите его в переменную `bias_min`. 

In [3]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

types_df = pd.DataFrame({ "type": df["type"], "diff": bias })
types_bias = types_df.groupby("type").mean()
types_bias["abs"] = types_bias["diff"].abs()
bias_min_row = types_bias[types_bias["abs"] == types_bias["abs"].min()]
bias_min = bias_min_row["diff"].item()

print("Тип с самым низким смещением: ", bias_min_row.index[0])

Тип с самым низким смещением:  presentation


In [ ]:
# проверка того, что задание решено верно
assert bias_min  + 1.47 < 1e-2

__в)__ Посмотрим как оцениватель Филиппа ведёт себя в динамике. Посчитайте среднее смещение для каждой недели.
Как оно ведёт себя в динамике? Правда ли, что оно постепенно сходится к нулю?
Чем поведение смещения в динамике отличается от асимптотической несмещённости? 

In [4]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

weeks_df = pd.DataFrame({ "week": df["week"], "diff": bias })
weeks_bias = weeks_df.groupby("week").mean()
weeks_bias

          diff
week          
1    -1.950000
2    -1.558333
3    -1.727273
4    -1.300000
5    -3.150000
6    -3.328571
7    -3.000000
8    -1.000000
9    -2.200000
10   -2.272727
11   -2.200000

Среднее смещение то уменьшается, то увеличивается, но никак не сходится к нулю. 

Асимптотическое смещение постепенно стремится к оцениваемому параметру и требует большого количества испытаний.
В данном случае испытаний мало, а среднее смещение не стремится к нулю.

__г)__ Несмещённость - это свойство при фиксированном $n$. Состоятельность - свойство при $n \to \infty$.
Представим себе, что Филипп переосмыслил свою стратегию оценки времени, которое он тратит на запись видео.
Теперь он хочет оценивать время на запись следующего видео как среднее, посчитанное по всем видео, записанным ранее.

_Например:_ Филипп записал три видео: 

- презентация 4 минуты
- презентация 1 минута
- доска 4 минуты

Тогда он оценить веремя на следующее видео как $\frac{4 + 1 + 4}{3} = 3$. Создайте в табличке колонку `new_estimate` с такми оценками.
Время на первое видео он оценивает как $6$ минут. Посмотрите как себя ведёт такая оценка в динамике.
Как думаете, к чему она сходится по вероятности? Какой физический смысл стоит за такой оценкой? 

In [5]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

df["new_estimate"] = df.apply(lambda x: 6 if x.name == 0 else df.iloc[0:x.name]["real_time"].mean(), axis=1)
df

     real_week  week          type  estimate_time  real_time  new_estimate
0            6     1    blackboard              6       11.0      6.000000
1            6     1    blackboard             10        7.0     11.000000
2            6     1    blackboard             10       11.5      9.000000
3            6     1    blackboard              5        6.0      9.833333
4            6     1  presentation              5        3.0      8.875000
..         ...   ...           ...            ...        ...           ...
124          4    11          code             10       13.0     10.448387
125          4    11          code             10       12.0     10.468800
126          4    11          code             10       14.0     10.480952
127          4    11          code             15       15.0     10.508661
128          4    11          code             10       11.5     10.543750

[129 rows x 6 columns]

При большом количестве испытаний такая оценка будет иметь минимальное смещение от реальный значений.

__д)__ Посчитайте для каждого видео смещение. По аналогии с пунктом __а)__ найдите среднее смещение.
Запишите его в переменную `bias_mean_new`. Стало ли оно меньше благодаря новому оценивателю? Почему? 

In [6]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

bias_new = df["new_estimate"] - df["real_time"]
bias_mean_new = bias_new.mean()
bias_mean_new

-0.6094253748224452

Новое среднее смещение стало меньше. Произошло это потому, что новая система
оценки стала учитывать реальные значения предыдущих испытаний.

In [ ]:
# проверка того, что задание решено верно
assert np.abs(bias_mean_new  + 0.609) < 1e-2

__е)__ Как Филипп мог бы использовать полученную в этом задании информацию, __чтобы стать лучше__?
Сформулируйте для Филиппа несколько дельных советов. 

Не брать оценку из головы, а использовать реальные данные.